In [1]:
# Source
datadownloadpath_Source = './datadownload/Prostate/BMC/'
datasavepath_Source = './datapreprocessed/Prostate/BMC/'
# Target
datadownloadpath_Target = './datadownload/Prostate/I2CVB/'
datasavepath_Target = './datapreprocessed/Prostate/I2CVB/'

In [2]:
import glob
import SimpleITK as sitk
import numpy as np
from scipy.ndimage import label, binary_fill_holes
import os

In [3]:
os.mkdir(datasavepath_Source)
os.mkdir(datasavepath_Target)
os.mkdir(datasavepath_Source + 'Image')
os.mkdir(datasavepath_Source + 'Label')
os.mkdir(datasavepath_Source + 'processed')
os.mkdir(datasavepath_Source + 'mask')
os.mkdir(datasavepath_Target + 'Image')
os.mkdir(datasavepath_Target + 'Label')
os.mkdir(datasavepath_Target + 'processed')
os.mkdir(datasavepath_Target + 'mask')

In [4]:
# helper functions copy pasted
def resample_by_res(mov_img_obj, new_spacing, interpolator = sitk.sitkLinear, logging = True):
    resample = sitk.ResampleImageFilter()
    resample.SetInterpolator(interpolator)
    resample.SetOutputDirection(mov_img_obj.GetDirection())
    resample.SetOutputOrigin(mov_img_obj.GetOrigin())
    resample.SetUseNearestNeighborExtrapolator(True)
    mov_spacing = mov_img_obj.GetSpacing()

    resample.SetOutputSpacing(new_spacing)
    RES_COE = np.array(mov_spacing) * 1.0 / np.array(new_spacing)
    new_size = np.array(mov_img_obj.GetSize()) *  RES_COE 

    resample.SetSize( [int(sz+1) for sz in new_size] )
    if logging:
        print("Spacing: {} -> {}".format(mov_spacing, new_spacing))
        print("Size {} -> {}".format( mov_img_obj.GetSize(), new_size ))

    return resample.Execute(mov_img_obj)

def resample_lb_by_res(mov_lb_obj, new_spacing, interpolator = sitk.sitkLinear, ref_img = None, logging = True):
    src_mat = sitk.GetArrayFromImage(mov_lb_obj)
    lbvs = np.unique(src_mat)
    if logging:
        print("Label values: {}".format(lbvs))
    for idx, lbv in enumerate(lbvs):
        _src_curr_mat = np.float32(src_mat == lbv) 
        _src_curr_obj = sitk.GetImageFromArray(_src_curr_mat)
        _src_curr_obj.CopyInformation(mov_lb_obj)
        _tar_curr_obj = resample_by_res( _src_curr_obj, new_spacing, interpolator, logging )
        _tar_curr_mat = np.rint(sitk.GetArrayFromImage(_tar_curr_obj)) * lbv
        if idx == 0:
            out_vol = _tar_curr_mat
        else:
            out_vol[_tar_curr_mat == lbv] = lbv
    out_obj = sitk.GetImageFromArray(out_vol)
    out_obj.SetSpacing( _tar_curr_obj.GetSpacing() )
    if ref_img != None:
        out_obj.CopyInformation(ref_img)
    return out_obj

# Source Date Preprocessing

In [5]:
IMG_FOLDER = datadownloadpath_Source
SEG_FOLDER = datadownloadpath_Source
imgs = glob.glob(IMG_FOLDER + "/Case??.nii.gz")
imgs = [ fid for fid in sorted(imgs) ]
segs = [ fid for fid in sorted(glob.glob(SEG_FOLDER + "/*Segmentation.nii.gz")) ]

pids = [pid.split("/")[-1].split(".")[0] for pid in imgs]

In [6]:
imgs

['./datadownload/Prostate/BMC/Case00.nii.gz',
 './datadownload/Prostate/BMC/Case01.nii.gz',
 './datadownload/Prostate/BMC/Case02.nii.gz',
 './datadownload/Prostate/BMC/Case03.nii.gz',
 './datadownload/Prostate/BMC/Case04.nii.gz',
 './datadownload/Prostate/BMC/Case05.nii.gz',
 './datadownload/Prostate/BMC/Case06.nii.gz',
 './datadownload/Prostate/BMC/Case07.nii.gz',
 './datadownload/Prostate/BMC/Case08.nii.gz',
 './datadownload/Prostate/BMC/Case09.nii.gz',
 './datadownload/Prostate/BMC/Case10.nii.gz',
 './datadownload/Prostate/BMC/Case11.nii.gz',
 './datadownload/Prostate/BMC/Case12.nii.gz',
 './datadownload/Prostate/BMC/Case13.nii.gz',
 './datadownload/Prostate/BMC/Case14.nii.gz',
 './datadownload/Prostate/BMC/Case15.nii.gz',
 './datadownload/Prostate/BMC/Case16.nii.gz',
 './datadownload/Prostate/BMC/Case17.nii.gz',
 './datadownload/Prostate/BMC/Case18.nii.gz',
 './datadownload/Prostate/BMC/Case19.nii.gz',
 './datadownload/Prostate/BMC/Case20.nii.gz',
 './datadownload/Prostate/BMC/Case

In [7]:
segs

['./datadownload/Prostate/BMC/Case00_Segmentation.nii.gz',
 './datadownload/Prostate/BMC/Case01_Segmentation.nii.gz',
 './datadownload/Prostate/BMC/Case02_Segmentation.nii.gz',
 './datadownload/Prostate/BMC/Case03_Segmentation.nii.gz',
 './datadownload/Prostate/BMC/Case04_Segmentation.nii.gz',
 './datadownload/Prostate/BMC/Case05_Segmentation.nii.gz',
 './datadownload/Prostate/BMC/Case06_Segmentation.nii.gz',
 './datadownload/Prostate/BMC/Case07_Segmentation.nii.gz',
 './datadownload/Prostate/BMC/Case08_Segmentation.nii.gz',
 './datadownload/Prostate/BMC/Case09_Segmentation.nii.gz',
 './datadownload/Prostate/BMC/Case10_Segmentation.nii.gz',
 './datadownload/Prostate/BMC/Case11_Segmentation.nii.gz',
 './datadownload/Prostate/BMC/Case12_Segmentation.nii.gz',
 './datadownload/Prostate/BMC/Case13_Segmentation.nii.gz',
 './datadownload/Prostate/BMC/Case14_Segmentation.nii.gz',
 './datadownload/Prostate/BMC/Case15_Segmentation.nii.gz',
 './datadownload/Prostate/BMC/Case16_Segmentation.nii.gz

In [8]:
pids

['Case00',
 'Case01',
 'Case02',
 'Case03',
 'Case04',
 'Case05',
 'Case06',
 'Case07',
 'Case08',
 'Case09',
 'Case10',
 'Case11',
 'Case12',
 'Case13',
 'Case14',
 'Case15',
 'Case16',
 'Case17',
 'Case18',
 'Case19',
 'Case20',
 'Case21',
 'Case22',
 'Case23',
 'Case24',
 'Case25',
 'Case26',
 'Case27',
 'Case28',
 'Case29']

## Step 1, Data resample

In [9]:
targetspacing = [0.8, 0.8, 1.5]

In [10]:
for (imgfile, lblfile, pid) in zip(imgs, segs, pids):
    savename = 'Prostate_' + pid + '.nii.gz'
    img_obj = sitk.ReadImage( imgfile )
    seg_obj = sitk.ReadImage( lblfile )
    img_spa_ori = img_obj.GetSpacing()
    seg_obj.SetSpacing(img_spa_ori)
    res_img_o = resample_by_res(img_obj, targetspacing, interpolator = sitk.sitkLinear,
                                    logging = True)
    res_lb_o = resample_lb_by_res(seg_obj, targetspacing, interpolator = sitk.sitkLinear,
                                  ref_img = res_img_o, logging = True)
    sitk.WriteImage(res_img_o, datasavepath_Source + 'Image/' + savename, True) 
    sitk.WriteImage(res_lb_o, datasavepath_Source + 'Label/' + savename, True) 

Spacing: (0.4166666865348816, 0.4166666865348816, 3.000002861022949) -> [0.8, 0.8, 1.5]
Size (384, 384, 32) -> [200.00000954 200.00000954  64.00006104]
Label values: [0. 1. 2.]
Spacing: (0.4166666865348816, 0.4166666865348816, 3.000002861022949) -> [0.8, 0.8, 1.5]
Size (384, 384, 32) -> [200.00000954 200.00000954  64.00006104]
Spacing: (0.4166666865348816, 0.4166666865348816, 3.000002861022949) -> [0.8, 0.8, 1.5]
Size (384, 384, 32) -> [200.00000954 200.00000954  64.00006104]
Spacing: (0.4166666865348816, 0.4166666865348816, 3.000002861022949) -> [0.8, 0.8, 1.5]
Size (384, 384, 32) -> [200.00000954 200.00000954  64.00006104]
Spacing: (0.4166666865348816, 0.4166666865348816, 2.999999523162842) -> [0.8, 0.8, 1.5]
Size (384, 384, 30) -> [200.00000954 200.00000954  59.99999046]
Label values: [0. 1. 2.]
Spacing: (0.4166666865348816, 0.4166666865348816, 2.999999523162842) -> [0.8, 0.8, 1.5]
Size (384, 384, 30) -> [200.00000954 200.00000954  59.99999046]
Spacing: (0.4166666865348816, 0.416666

Spacing: (0.4166666865348816, 0.4166666865348816, 3.000000238418579) -> [0.8, 0.8, 1.5]
Size (384, 384, 32) -> [200.00000954 200.00000954  64.00000509]
Label values: [0. 1. 2.]
Spacing: (0.4166666865348816, 0.4166666865348816, 3.000000238418579) -> [0.8, 0.8, 1.5]
Size (384, 384, 32) -> [200.00000954 200.00000954  64.00000509]
Spacing: (0.4166666865348816, 0.4166666865348816, 3.000000238418579) -> [0.8, 0.8, 1.5]
Size (384, 384, 32) -> [200.00000954 200.00000954  64.00000509]
Spacing: (0.4166666865348816, 0.4166666865348816, 3.000000238418579) -> [0.8, 0.8, 1.5]
Size (384, 384, 32) -> [200.00000954 200.00000954  64.00000509]
Spacing: (0.4166666865348816, 0.4166666865348816, 2.9999990463256836) -> [0.8, 0.8, 1.5]
Size (384, 384, 32) -> [200.00000954 200.00000954  63.99997965]
Label values: [0. 1. 2.]
Spacing: (0.4166666865348816, 0.4166666865348816, 2.9999990463256836) -> [0.8, 0.8, 1.5]
Size (384, 384, 32) -> [200.00000954 200.00000954  63.99997965]
Spacing: (0.4166666865348816, 0.4166

Spacing: (0.4166666865348816, 0.4166666865348816, 3.000000238418579) -> [0.8, 0.8, 1.5]
Size (384, 384, 34) -> [200.00000954 200.00000954  68.0000054 ]
Label values: [0. 1. 2.]
Spacing: (0.4166666865348816, 0.4166666865348816, 3.000000238418579) -> [0.8, 0.8, 1.5]
Size (384, 384, 34) -> [200.00000954 200.00000954  68.0000054 ]
Spacing: (0.4166666865348816, 0.4166666865348816, 3.000000238418579) -> [0.8, 0.8, 1.5]
Size (384, 384, 34) -> [200.00000954 200.00000954  68.0000054 ]
Spacing: (0.4166666865348816, 0.4166666865348816, 3.000000238418579) -> [0.8, 0.8, 1.5]
Size (384, 384, 34) -> [200.00000954 200.00000954  68.0000054 ]
Spacing: (0.4166666865348816, 0.4166666865348816, 3.0000059604644775) -> [0.8, 0.8, 1.5]
Size (384, 384, 32) -> [200.00000954 200.00000954  64.00012716]
Label values: [0. 1. 2.]
Spacing: (0.4166666865348816, 0.4166666865348816, 3.0000059604644775) -> [0.8, 0.8, 1.5]
Size (384, 384, 32) -> [200.00000954 200.00000954  64.00012716]
Spacing: (0.4166666865348816, 0.4166

## Step 2, Intensity Normalization

In [11]:
IMG_FOLDER_SAVE = datasavepath_Source + 'Image/'
imgs = glob.glob(IMG_FOLDER_SAVE + "/Prostate_*.nii.gz")
imgs = [ fid for fid in sorted(imgs) ]

In [12]:
for (imgfile, pid) in zip(imgs, pids):
    savename = 'Prostate_' + pid + '.nii.gz'
    img_obj = sitk.ReadImage( imgfile )
    
    array = sitk.GetArrayFromImage(img_obj)

    pixel_mean = np.mean(array)
    pixel_std = np.std(array)
    array = (array - pixel_mean) / pixel_std

    normalized_img = sitk.GetImageFromArray(array)
    normalized_img.CopyInformation(img_obj)
    
    sitk.WriteImage(normalized_img, datasavepath_Source + 'processed/' + savename, True) 
    print('Id ' + pid + ' normalized')

Id Case00 normalized
Id Case01 normalized
Id Case02 normalized
Id Case03 normalized
Id Case04 normalized
Id Case05 normalized
Id Case06 normalized
Id Case07 normalized
Id Case08 normalized
Id Case09 normalized
Id Case10 normalized
Id Case11 normalized
Id Case12 normalized
Id Case13 normalized
Id Case14 normalized
Id Case15 normalized
Id Case16 normalized
Id Case17 normalized
Id Case18 normalized
Id Case19 normalized
Id Case20 normalized
Id Case21 normalized
Id Case22 normalized
Id Case23 normalized
Id Case24 normalized
Id Case25 normalized
Id Case26 normalized
Id Case27 normalized
Id Case28 normalized
Id Case29 normalized


## Step 3, Generate Binary Segmentation Masks

In [13]:
SEG_FOLDER_SAVE = datasavepath_Source + 'Label/'
segs = [ fid for fid in sorted(glob.glob(SEG_FOLDER_SAVE + "/Prostate_*.nii.gz")) ]

In [14]:
for (segfile, pid) in zip(segs, pids):
    savename = 'Prostate_' + pid + '.nii.gz'
    seg_obj = sitk.ReadImage( segfile )
    
    array = sitk.GetArrayFromImage(seg_obj)

    array[array > 0] = 1

    binary_seg = sitk.GetImageFromArray(array)
    binary_seg.CopyInformation(seg_obj)
    
    sitk.WriteImage(binary_seg, datasavepath_Source + 'Label/' + savename, True) 
    print('Id ' + pid + ' generated')

Id Case00 generated
Id Case01 generated
Id Case02 generated
Id Case03 generated
Id Case04 generated
Id Case05 generated
Id Case06 generated
Id Case07 generated
Id Case08 generated
Id Case09 generated
Id Case10 generated
Id Case11 generated
Id Case12 generated
Id Case13 generated
Id Case14 generated
Id Case15 generated
Id Case16 generated
Id Case17 generated
Id Case18 generated
Id Case19 generated
Id Case20 generated
Id Case21 generated
Id Case22 generated
Id Case23 generated
Id Case24 generated
Id Case25 generated
Id Case26 generated
Id Case27 generated
Id Case28 generated
Id Case29 generated


## Step 4, Generate Body Masks

In [15]:
IMG_FOLDER_SAVE = datasavepath_Source + 'Image/'
imgs = glob.glob(IMG_FOLDER_SAVE + "/Prostate_*.nii.gz")
imgs = [ fid for fid in sorted(imgs) ]

In [16]:
for (imgfile, pid) in zip(imgs, pids):
    savename = 'Prostate_' + pid + '.nii.gz'
    img_obj = sitk.ReadImage( imgfile )
    array = sitk.GetArrayFromImage(img_obj)
    array_mask = array > -999
    array_mask = array_mask.astype(float)

    mask_img = sitk.GetImageFromArray(array_mask)
    mask_img.CopyInformation(img_obj)
    
    sitk.WriteImage(mask_img, datasavepath_Source + 'mask/' + savename, True) 

# Target Data Preprocessing

In [17]:
IMG_FOLDER = datadownloadpath_Target
SEG_FOLDER = datadownloadpath_Target
imgs = glob.glob(IMG_FOLDER + "/Case??.nii.gz")
imgs = [ fid for fid in sorted(imgs) ]
segs = [ fid for fid in sorted(glob.glob(SEG_FOLDER + "/*segmentation.nii.gz")) ]

pids = [pid.split("/")[-1].split(".")[0] for pid in imgs]

In [18]:
imgs

['./datadownload/Prostate/I2CVB/Case00.nii.gz',
 './datadownload/Prostate/I2CVB/Case01.nii.gz',
 './datadownload/Prostate/I2CVB/Case02.nii.gz',
 './datadownload/Prostate/I2CVB/Case03.nii.gz',
 './datadownload/Prostate/I2CVB/Case04.nii.gz',
 './datadownload/Prostate/I2CVB/Case05.nii.gz',
 './datadownload/Prostate/I2CVB/Case06.nii.gz',
 './datadownload/Prostate/I2CVB/Case07.nii.gz',
 './datadownload/Prostate/I2CVB/Case08.nii.gz',
 './datadownload/Prostate/I2CVB/Case09.nii.gz',
 './datadownload/Prostate/I2CVB/Case10.nii.gz',
 './datadownload/Prostate/I2CVB/Case11.nii.gz',
 './datadownload/Prostate/I2CVB/Case12.nii.gz',
 './datadownload/Prostate/I2CVB/Case13.nii.gz',
 './datadownload/Prostate/I2CVB/Case14.nii.gz',
 './datadownload/Prostate/I2CVB/Case15.nii.gz',
 './datadownload/Prostate/I2CVB/Case16.nii.gz',
 './datadownload/Prostate/I2CVB/Case17.nii.gz',
 './datadownload/Prostate/I2CVB/Case18.nii.gz']

In [19]:
segs

['./datadownload/Prostate/I2CVB/Case00_segmentation.nii.gz',
 './datadownload/Prostate/I2CVB/Case01_segmentation.nii.gz',
 './datadownload/Prostate/I2CVB/Case02_segmentation.nii.gz',
 './datadownload/Prostate/I2CVB/Case03_segmentation.nii.gz',
 './datadownload/Prostate/I2CVB/Case04_segmentation.nii.gz',
 './datadownload/Prostate/I2CVB/Case05_segmentation.nii.gz',
 './datadownload/Prostate/I2CVB/Case06_segmentation.nii.gz',
 './datadownload/Prostate/I2CVB/Case07_segmentation.nii.gz',
 './datadownload/Prostate/I2CVB/Case08_segmentation.nii.gz',
 './datadownload/Prostate/I2CVB/Case09_segmentation.nii.gz',
 './datadownload/Prostate/I2CVB/Case10_segmentation.nii.gz',
 './datadownload/Prostate/I2CVB/Case11_segmentation.nii.gz',
 './datadownload/Prostate/I2CVB/Case12_segmentation.nii.gz',
 './datadownload/Prostate/I2CVB/Case13_segmentation.nii.gz',
 './datadownload/Prostate/I2CVB/Case14_segmentation.nii.gz',
 './datadownload/Prostate/I2CVB/Case15_segmentation.nii.gz',
 './datadownload/Prostat

In [20]:
pids

['Case00',
 'Case01',
 'Case02',
 'Case03',
 'Case04',
 'Case05',
 'Case06',
 'Case07',
 'Case08',
 'Case09',
 'Case10',
 'Case11',
 'Case12',
 'Case13',
 'Case14',
 'Case15',
 'Case16',
 'Case17',
 'Case18']

## Step 1, Data resample

In [21]:
targetspacing = [0.8, 0.8, 1.5]

In [22]:
for (imgfile, lblfile, pid) in zip(imgs, segs, pids):
    savename = 'Prostate_' + pid + '.nii.gz'
    img_obj = sitk.ReadImage( imgfile )
    seg_obj = sitk.ReadImage( lblfile )
    img_spa_ori = img_obj.GetSpacing()
    seg_obj.SetSpacing(img_spa_ori)
    res_img_o = resample_by_res(img_obj, targetspacing, interpolator = sitk.sitkLinear,
                                    logging = True)
    res_lb_o = resample_lb_by_res(seg_obj, targetspacing, interpolator = sitk.sitkLinear,
                                  ref_img = res_img_o, logging = True)
    sitk.WriteImage(res_img_o, datasavepath_Target + 'Image/' + savename, True) 
    sitk.WriteImage(res_lb_o, datasavepath_Target + 'Label/' + savename, True) 

Spacing: (0.5342610478401184, 0.42852187156677246, 1.2500853538513184) -> [0.8, 0.8, 1.5]
Size (384, 384, 64) -> [256.44530296 205.69049835  53.3369751 ]
Label values: [0. 1.]
Spacing: (0.5342610478401184, 0.42852187156677246, 1.2500853538513184) -> [0.8, 0.8, 1.5]
Size (384, 384, 64) -> [256.44530296 205.69049835  53.3369751 ]
Spacing: (0.5342610478401184, 0.42852187156677246, 1.2500853538513184) -> [0.8, 0.8, 1.5]
Size (384, 384, 64) -> [256.44530296 205.69049835  53.3369751 ]
Spacing: (0.5342610478401184, 0.42852187156677246, 1.2499675750732422) -> [0.8, 0.8, 1.5]
Size (384, 384, 64) -> [256.44530296 205.69049835  53.33194987]
Label values: [0. 1.]
Spacing: (0.5342610478401184, 0.42852187156677246, 1.2499675750732422) -> [0.8, 0.8, 1.5]
Size (384, 384, 64) -> [256.44530296 205.69049835  53.33194987]
Spacing: (0.5342610478401184, 0.42852187156677246, 1.2499675750732422) -> [0.8, 0.8, 1.5]
Size (384, 384, 64) -> [256.44530296 205.69049835  53.33194987]
Spacing: (0.5342610478401184, 0.

Label values: [0. 1.]
Spacing: (0.5342610478401184, 0.4388573169708252, 1.249906063079834) -> [0.8, 0.8, 1.5]
Size (384, 384, 64) -> [256.44530296 210.65151215  53.32932536]
Spacing: (0.5342610478401184, 0.4388573169708252, 1.249906063079834) -> [0.8, 0.8, 1.5]
Size (384, 384, 64) -> [256.44530296 210.65151215  53.32932536]
Spacing: (0.6171332597732544, 0.5069308876991272, 1.2498283386230469) -> [0.8, 0.8, 1.5]
Size (384, 384, 64) -> [296.22396469 243.3268261   53.32600911]
Label values: [0. 1.]
Spacing: (0.6171332597732544, 0.5069308876991272, 1.2498283386230469) -> [0.8, 0.8, 1.5]
Size (384, 384, 64) -> [296.22396469 243.3268261   53.32600911]
Spacing: (0.6171332597732544, 0.5069308876991272, 1.2498283386230469) -> [0.8, 0.8, 1.5]
Size (384, 384, 64) -> [296.22396469 243.3268261   53.32600911]


## Step 2, Intensity Normalization

In [23]:
IMG_FOLDER_SAVE = datasavepath_Target + 'Image/'
imgs = glob.glob(IMG_FOLDER_SAVE + "/Prostate_*.nii.gz")
imgs = [ fid for fid in sorted(imgs) ]

In [24]:
for (imgfile, pid) in zip(imgs, pids):
    savename = 'Prostate_' + pid + '.nii.gz'
    img_obj = sitk.ReadImage( imgfile )
    
    array = sitk.GetArrayFromImage(img_obj)

    pixel_mean = np.mean(array)
    pixel_std = np.std(array)
    array = (array - pixel_mean) / pixel_std

    normalized_img = sitk.GetImageFromArray(array)
    normalized_img.CopyInformation(img_obj)
    
    sitk.WriteImage(normalized_img, datasavepath_Target + 'processed/' + savename, True) 
    print('Id ' + pid + ' normalized')

Id Case00 normalized
Id Case01 normalized
Id Case02 normalized
Id Case03 normalized
Id Case04 normalized
Id Case05 normalized
Id Case06 normalized
Id Case07 normalized
Id Case08 normalized
Id Case09 normalized
Id Case10 normalized
Id Case11 normalized
Id Case12 normalized
Id Case13 normalized
Id Case14 normalized
Id Case15 normalized
Id Case16 normalized
Id Case17 normalized
Id Case18 normalized


## Step 3, Generate Binary Segmentation Masks

In [25]:
SEG_FOLDER_SAVE = datasavepath_Target + 'Label/'
segs = [ fid for fid in sorted(glob.glob(SEG_FOLDER_SAVE + "/Prostate_*.nii.gz")) ]

In [26]:
for (segfile, pid) in zip(segs, pids):
    savename = 'Prostate_' + pid + '.nii.gz'
    seg_obj = sitk.ReadImage( segfile )
    
    array = sitk.GetArrayFromImage(seg_obj)

    array[array > 0] = 1

    binary_seg = sitk.GetImageFromArray(array)
    binary_seg.CopyInformation(seg_obj)
    
    sitk.WriteImage(binary_seg, datasavepath_Target + 'Label/' + savename, True) 
    print('Id ' + pid + ' generated')

Id Case00 generated
Id Case01 generated
Id Case02 generated
Id Case03 generated
Id Case04 generated
Id Case05 generated
Id Case06 generated
Id Case07 generated
Id Case08 generated
Id Case09 generated
Id Case10 generated
Id Case11 generated
Id Case12 generated
Id Case13 generated
Id Case14 generated
Id Case15 generated
Id Case16 generated
Id Case17 generated
Id Case18 generated


## Step 4, Generate Body Masks

In [27]:
IMG_FOLDER_SAVE = datasavepath_Target + 'Image/'
imgs = glob.glob(IMG_FOLDER_SAVE + "/Prostate_*.nii.gz")
imgs = [ fid for fid in sorted(imgs) ]

In [28]:
for (imgfile, pid) in zip(imgs, pids):
    savename = 'Prostate_' + pid + '.nii.gz'
    img_obj = sitk.ReadImage( imgfile )
    array = sitk.GetArrayFromImage(img_obj)
    array_mask = array > -999
    array_mask = array_mask.astype(float)

    mask_img = sitk.GetImageFromArray(array_mask)
    mask_img.CopyInformation(img_obj)
    
    sitk.WriteImage(mask_img, datasavepath_Target + 'mask/' + savename, True) 